<a href="https://colab.research.google.com/github/ElenaMorozzova/NLP_Intro/blob/main/NLP%20DZ9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
!pip install stop_words

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import tensorflow as tf
import os

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 9.0 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=59cfcf192848d37da020671919e8148c4c80aefdb64b1e419af58ff7ae070e31
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving eminem_lyrics.txt to eminem_lyrics.txt


**Load and preprocess text**

In [ ]:
text = open('eminem_lyrics.txt', 'r').read()
print('Length of text: {} characters'.format(len(text)))

Length of text: 439328 characters


In [ ]:
text = text.split('\n\n')
text[:5]

['"Walk On Water"\n(feat. Beyoncé)',
 "[Beyoncé (Eminem):]\nI walk on water\nBut I ain't no Jesus\nI walk on water\nBut only when it freezes (fuck)",
 "[Eminem:]\nWhy are expectations so high?\nIs it the bar I set?\nMy arms, I stretch, but I can't reach\nA far cry from it, or it's in my grasp, but as\nSoon as I grab, squeeze\nI lose my grip like the flyin' trapeze\nInto the dark I plummet, now the sky's blackenin'\nI know the mark's high, butterflies rip apart my stomach\nKnowin' that no matter what bars I come with\nYou're gonna harp, gripe, and\nThat's a hard Vicodin to swallow, so I scrap these\nAs pressure increases like khakis\nI feel the ice cracking, because",
 "[Beyoncé (Eminem):]\nI walk on water\nBut I ain't no Jesus\n(It's the curse of the, it's the curse of the)\nI walk on water (shit)\nBut only when it freezes",
 '[Eminem:]\nIt\'s the curse of the standard, that the first of the Mathers disc set\nAlways in search of the verse that I haven\'t spit yet\nWill this step just b

In [ ]:
text_only = []
for strofa in text:
    if len(strofa) < 350:
        continue
    else:
        text_only.append(strofa)
len(text_only)

356

In [ ]:
text_only[:5]

["[Eminem:]\nWhy are expectations so high?\nIs it the bar I set?\nMy arms, I stretch, but I can't reach\nA far cry from it, or it's in my grasp, but as\nSoon as I grab, squeeze\nI lose my grip like the flyin' trapeze\nInto the dark I plummet, now the sky's blackenin'\nI know the mark's high, butterflies rip apart my stomach\nKnowin' that no matter what bars I come with\nYou're gonna harp, gripe, and\nThat's a hard Vicodin to swallow, so I scrap these\nAs pressure increases like khakis\nI feel the ice cracking, because",
 '[Eminem:]\nIt\'s the curse of the standard, that the first of the Mathers disc set\nAlways in search of the verse that I haven\'t spit yet\nWill this step just be another misstep\nTo tarnish, whatever the legacy, love or respect\nI\'ve garnered?\nThe rhyme has to be perfect, the delivery flawless\nAnd it always feels like I\'m hittin\' the mark\n\'Til I go sit in the car, listen and pick it apart\nLike, "This shit is garbage!"\nGod\'s given me all this, still I feel n

In [ ]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "Content"})
data

,Content
0,[Eminem:]\nWhy are expectations so high?\nIs i...
1,"[Eminem:]\nIt's the curse of the standard, tha..."
2,"[Eminem:]\nIt's true, I'm a Rubik's, a beautif..."
3,I want you to understand something\nThat when ...
4,This is when shit hits the fan like it just sp...
...,...
351,"Some boys are wiling\nDrinkin', cussin', and h..."
352,Yeah this game has got me goin' crazy\nFuck it...
353,Got a dick as big as a banana\nI try to contro...
354,Are you ready? You better hold the camera stea...


In [ ]:
sw = set(get_stop_words("en"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)    
    txt = re.sub("\n", " \n", txt)    
    return txt

def preprocess_text(txt, morph = False):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\n", "zzz", txt)
    new_txt =[]
    for word in txt.split():
        if word == "zzz":
            word = " \n"
            
        else:
            if morph:
                word = morpher.parse(word)[0].normal_form
            else:
                pass
        new_txt.append(word)

    return new_txt

data['Content_splited'] = data['Content'].apply(exclude_punctuation)
data['Content_splited_morph'] = data['Content_splited'].apply(preprocess_text, morph = True)
data['Content_splited'] = data['Content_splited'].apply(preprocess_text, morph = False)

data.head(10)

,Content,Content_splited,Content_splited_morph
0,[Eminem:]\nWhy are expectations so high?\nIs i...,"[eminem, zzzwhy, are, expectations, so, high, ...","[eminem, zzzwhy, are, expectations, so, high, ..."
1,"[Eminem:]\nIt's the curse of the standard, tha...","[eminem, zzzits, the, curse, of, the, standard...","[eminem, zzzits, the, curse, of, the, standard..."
2,"[Eminem:]\nIt's true, I'm a Rubik's, a beautif...","[eminem, zzzits, true, im, a, rubiks, a, beaut...","[eminem, zzzits, true, im, a, rubiks, a, beaut..."
3,I want you to understand something\nThat when ...,"[i, want, you, to, understand, something, zzzt...","[i, want, you, to, understand, something, zzzt..."
4,This is when shit hits the fan like it just sp...,"[this, is, when, shit, hits, the, fan, like, i...","[this, is, when, shit, hits, the, fan, like, i..."
5,"Now I know you're feeling discouraged\nBut, ho...","[now, i, know, youre, feeling, discouraged, zz...","[now, i, know, youre, feeling, discouraged, zz..."
6,"Yeah, so let us in 'fore we huff and puff and ...","[yeah, so, let, us, in, fore, we, huff, and, p...","[yeah, so, let, us, in, fore, we, huff, and, p..."
7,If I was frozen inside of a moment\nIf I could...,"[if, i, was, frozen, inside, of, a, moment, zz...","[if, i, was, frozen, inside, of, a, moment, zz..."
8,Lose yourself in this music\nThis moment we ow...,"[lose, yourself, in, this, music, zzzthis, mom...","[lose, yourself, in, this, music, zzzthis, mom..."
9,"Cause when we descend together, we begin to mo...","[cause, when, we, descend, together, we, begin...","[cause, when, we, descend, together, we, begin..."


In [ ]:
def get_int(column_data):
    
    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))
    print('{} unique characters'.format(len(vocab)))            
            
    # Creating a mapping from unique characters to indices
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    
    print(len(vocab), len(char2idx), len(idx2char))
    return char2idx, idx2char, vocab

In [ ]:
char2idx, idx2char, vocab = get_int(data['Content_splited'])        
data['text_as_int'] = data['Content_splited'].apply(lambda x: [char2idx[c] for c in x])

9414 unique characters
9414 9414 9414


In [ ]:
data.head()

,Content,Content_splited,Content_splited_morph,text_as_int
0,[Eminem:]\nWhy are expectations so high?\nIs i...,"[eminem, zzzwhy, are, expectations, so, high, ...","[eminem, zzzwhy, are, expectations, so, high, ...","[2359, 9362, 338, 2486, 6542, 3363, 8670, 3734..."
1,"[Eminem:]\nIt's the curse of the standard, tha...","[eminem, zzzits, the, curse, of, the, standard...","[eminem, zzzits, the, curse, of, the, standard...","[2359, 8675, 7176, 1749, 4897, 7176, 6751, 717..."
2,"[Eminem:]\nIt's true, I'm a Rubik's, a beautif...","[eminem, zzzits, true, im, a, rubiks, a, beaut...","[eminem, zzzits, true, im, a, rubiks, a, beaut...","[2359, 8675, 7441, 3594, 52, 5952, 52, 596, 44..."
3,I want you to understand something\nThat when ...,"[i, want, you, to, understand, something, zzzt...","[i, want, you, to, understand, something, zzzt...","[3567, 7758, 8080, 7295, 7540, 6578, 9224, 785..."
4,This is when shit hits the fan like it just sp...,"[this, is, when, shit, hits, the, fan, like, i...","[this, is, when, shit, hits, the, fan, like, i...","[7212, 3723, 7852, 6276, 3395, 7176, 2580, 412..."


In [ ]:
def get_all_int(column_data):
    
    int_dump = column_data.values
    all_txt_as_int = []

    for sublist in int_dump:
        for item in sublist:
            all_txt_as_int.append(item)
    all_txt_as_int = np.array(all_txt_as_int)    
    
    return all_txt_as_int

all_txt_as_int = get_all_int(data['text_as_int'])
all_txt_as_int

array([2359, 9362,  338, ...,  715, 3871, 3042])

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 35
examples_per_epoch = len(all_txt_as_int)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(all_txt_as_int)

for i in word_dataset.take(15):
    print(idx2char[i.numpy()])

eminem
zzzwhy
are
expectations
so
high
zzzis
it
the
bar
i
set
zzzmy
arms
i


In [ ]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(3):
    print(repr(idx2char[item.numpy()]))

array(['eminem', 'zzzwhy', 'are', 'expectations', 'so', 'high', 'zzzis',
       'it', 'the', 'bar', 'i', 'set', 'zzzmy', 'arms', 'i', 'stretch',
       'but', 'i', 'cant', 'reach', 'zzza', 'far', 'cry', 'from', 'it',
       'or', 'its', 'in', 'my', 'grasp', 'but', 'as', 'zzzsoon', 'as',
       'i', 'grab'], dtype='<U28')
array(['squeeze', 'zzzi', 'lose', 'my', 'grip', 'like', 'the', 'flyin',
       'trapeze', 'zzzinto', 'the', 'dark', 'i', 'plummet', 'now', 'the',
       'skys', 'blackenin', 'zzzi', 'know', 'the', 'marks', 'high',
       'butterflies', 'rip', 'apart', 'my', 'stomach', 'zzzknowin',
       'that', 'no', 'matter', 'what', 'bars', 'i', 'come'], dtype='<U28')
array(['with', 'zzzyoure', 'gonna', 'harp', 'gripe', 'and', 'zzzthats',
       'a', 'hard', 'vicodin', 'to', 'swallow', 'so', 'i', 'scrap',
       'these', 'zzzas', 'pressure', 'increases', 'like', 'khakis',
       'zzzi', 'feel', 'the', 'ice', 'cracking', 'because', 'eminem',
       'zzzits', 'the', 'curse', 'of', 'th

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  'eminem zzzwhy are expectations so high zzzis it the bar i set zzzmy arms i stretch but i cant reach zzza far cry from it or its in my grasp but as zzzsoon as i'
Target data: 'zzzwhy are expectations so high zzzis it the bar i set zzzmy arms i stretch but i cant reach zzza far cry from it or its in my grasp but as zzzsoon as i grab'


Build model

In [ ]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 35), (256, 35)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [ ]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
 
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 35, 9414) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         2409984   
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 9414)        9649350   
                                                                 
Total params: 28,592,838
Trainable params: 28,592,838
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (256, 35, 9414)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.14996


In [ ]:
model.compile(optimizer='adam', loss=loss)

**Set checkpoints and train the model**

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=20,
    save_weights_only=True)

In [ ]:
EPOCHS = 50

In [ ]:
for i in range(EPOCHS):
    model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])
    #model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
    model.reset_states()

7/7 [==============================] - 204s 29s/step - loss: 6.6940


Model loss decreasing very slow, 50 epochs were trained > 3 hours.

Generate text

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_1'

In [51]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 30

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ' '.join(text_generated))

In [57]:
print(generate_text(model, start_string="k"))

kon to a in was to to to in aint my for zzzbut i the in and you never the the you the from up to you a a skinning


In [62]:
print(generate_text(model, start_string="n"))

nzzzwait right the the the of ok this dimension mothafuckers zzzdont plastic a spider to get to the is like in me go a of ready of in you of
